In [1]:
#import openslide
from stardist.models import StarDist2D
from csbdeep.utils import Path, normalize
# impoirt resize from skimage
from skimage.transform import resize
# import plt
import matplotlib.pyplot as plt

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#import cv2
import os
import pandas as pd
from skimage.color import rgb2hsv, rgb2gray, rgb2hed
import matplotlib.patches as patches
from skimage.filters import gaussian
from skimage import measure
from skimage import io as skio
from skimage.registration import phase_cross_correlation
from skimage.color import rgb2gray

In [3]:
def find_max(contours):
  bbox = (0,0,0,0)
  max_area = 0
  for contour in contours:
    Xmin = np.min(contour[:,0])
    Xmax = np.max(contour[:,0])
    Ymin = np.min(contour[:,1])
    Ymax = np.max(contour[:,1])
    area = (Xmax-Xmin)*(Ymax-Ymin)
    if area > max_area:
      max_area = area
      bbox = (Xmin, Xmax, Ymin, Ymax)
  print(max_area)
  return bbox

In [4]:
def plot_max(bbox,im):
  (Xmin, Xmax, Ymin, Ymax) = bbox
  # Create figure and axes
  fig, ax = plt.subplots()
  # Display the image
  ax.imshow(im)
  # Create a Rectangle patch
  rect = patches.Rectangle((Ymin, Xmin), Ymax-Ymin, Xmax-Xmin, linewidth=1, edgecolor='b', facecolor='none')
  # Add the patch to the Axes
  ax.add_patch(rect)
  plt.show()

In [ ]:
def get_mask(roi,roi_he, num, case_name):
  ihc_hed = rgb2hed(roi)
  opened = ihc_hed[:,:,2]>0.02
  opened_3 = np.stack((opened,opened,opened),axis=-1)*255
  skio.imsave('./data_output/ver4_test_fine/{}_thresh.png'.format(case_name), opened_3[:,:,0].astype('uint8'))
  
  fig, (ax0, ax1, ax2, ax3) = plt.subplots(ncols=4, figsize=(20, 5))
  ax0.imshow(roi_he.astype('uint8'))
  ax1.imshow((roi).astype('uint8'), cmap='gray')
  ax2.imshow((opened_3).astype('uint8'), cmap='gray')
  ax3.imshow((opened_3*0.3).astype('uint8')+(roi_he*0.7).astype('uint8'), cmap='gray')
  ax0.axis('off')
  ax1.axis('off')
  ax2.axis('off')
  ax3.axis('off')
  fig.tight_layout()
  plt.title("Tile {}".format(case_name))
  plt.show()

In [ ]:
# read WSI
case_name = 'Geomix Slide 1'
fixed_wsi = "./data_geomix/wsis/{} H&E.svs".format(case_name)
float_wsi = "./data_geomix/wsis/{} CD3-CD20.svs".format(case_name)
fixed_wsi_obj = openslide.open_slide(fixed_wsi)
float_wsi_obj = openslide.open_slide(float_wsi)
fix_level = fixed_wsi_obj.level_dimensions
print(fix_level)
float_level = float_wsi_obj.level_dimensions
print(float_level)
he = fixed_wsi_obj.read_region((0,0),2,fix_level[2])
ihc = float_wsi_obj.read_region((0,0),2,float_level[2])
he = np.array(he)[:,:,:3].astype('uint8')
ihc = np.array(ihc)[:,:,:3].astype('uint8')

# find bbox of H&E biopsy sample at a low resolution
filtered_he = gaussian(he, sigma=3)
hsv_img = rgb2hsv(filtered_he)
hue_img = hsv_img[:,:,0]
thresh_img = (hue_img>0)*1.0
contours = measure.find_contours(thresh_img,0.5)
bbox1 = find_max(contours)
plot_max(bbox1,he)

# find bbox of IHC
filtered_ihc = gaussian(ihc, sigma=6)
hsv_img = rgb2hsv(filtered_ihc)
hue_img = hsv_img[:,:,0]
thresh_img = (hue_img>0)*1.0
contours = measure.find_contours(thresh_img)
bbox2 = find_max(contours)
plot_max(bbox2,ihc)

# coarse registration
roffset = int((bbox2[0]+bbox2[1])/2 - (bbox1[0]+bbox1[1])/2) * 16
coffset = int((bbox2[2]+bbox2[3])/2 - (bbox1[2]+bbox1[3])/2) * 16

# read patch locations
df = pd.read_excel('./Tile_xy.xlsx')
df1 = df[df['Case']==case_name]
actual_Xs = list(df1['X'])
actual_Ys = list(df1['Y'])
patch_size = 1024

for ind, (actual_X, actual_Y) in enumerate(zip(actual_Xs, actual_Ys)): 
    # get paired location after coarse registration
    rr = int(actual_Y*3.82- patch_size/2 - roffset)
    cc = int(actual_X*3.82- patch_size/2 - coffset)

    # get WSI region on HE and IHC for fine register
    fixed_patch = fixed_wsi_obj.read_region((cc,rr),0,(patch_size, patch_size))
    float_patch = float_wsi_obj.read_region((cc+coffset,rr+roffset),0,(patch_size, patch_size))

    # fine register
    image = np.array(fixed_patch)[:,:,:3]
    offset_image = np.array(float_patch)[:,:,:3]
    shift, error, diffphase = phase_cross_correlation(image, offset_image)
    print(f'Tile {ind} Detected pixel offset (y, x): {shift}')

    # get corrected IHC region for save
    float_patch = float_wsi_obj.read_region((cc+coffset-int(shift[1]),rr+roffset-int(shift[0])),0,(patch_size, patch_size))
    fixed_patch = np.array(fixed_patch)[:,:,:3]
    float_patch = np.array(float_patch)[:,:,:3]

    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(8, 4))
    ax0.imshow(fixed_patch)
    ax1.imshow(float_patch)
    plt.show()
    
    if not os.path.isdir('./data_geomix/pairs/{}'.format(case_name)):
        os.mkdir('./data_geomix/pairs/{}'.format(case_name))
    skio.imsave("./data_geomix/pairs/{}/p{}_he.png".format(case_name, ind),fixed_patch.astype('uint8'))
    skio.imsave("./data_geomix/pairs/{}/p{}_ihc.png".format(case_name, ind),float_patch.astype('uint8'))

In [ ]:
# nuclei segment model
model = StarDist2D.from_pretrained('2D_versatile_he')

In [ ]:
# get nuclei mask
names = os.listdir('./data_geomix/pairs')
for name in names:
    he = skio.imread('./data_geomix/pairs/{}/p0_he.png'.format(name))
  
    axis_norm = (0,1,2)
    img = normalize(he, 1,99.8, axis=axis_norm)
    
    output = model.predict(img)

    io = resize(output[0], (output[0].shape[0]*2, output[0].shape[1]*2), anti_aliasing=True)
    io = io*255

    iol = measure.label(io>50)
    rps = measure.regionprops(iol)

    id_ecc = np.where(np.asarray([r.eccentricity for r in rps])<.95)[0]+1
    id_area = np.where((np.asarray([r.area for r in rps])>30) )[0]+1
    mask1 = np.isin(iol,id_ecc)
    mask2 = np.isin(iol,id_area)

    mega = mask1 & mask2
    plt.imshow(mega)
    plt.show()
    skio.imsave('./data_geomix/pairs/{}/p0_nuclei.png'.format(name), (mega*255).astype('uint8'))

In [ ]:
# get ihc threshold
names = os.listdir('./data_output/ver4_test_fine')
for name in names:
    if 'he' in name:
      name = name[:-7]
      roi = skio.imread('./data_output/ver4_test_fine/{}_ihc.png'.format(name))
      roi_he = skio.imread('./data_output/ver4_test_fine/{}_he.png'.format(name))
      get_mask(roi,roi_he, 0, name)
      